In [2]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
from math import sin, cos, sqrt, atan2, radians
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
from datetime import timedelta
import datetime as dt
import sklearn.model_selection as ms
from keras.layers.convolutional import Convolution2D, Convolution3D
from keras.models import Sequential
from keras.layers import Dense

import keras



In [3]:
train = pd.read_csv('/content/drive/MyDrive/Dataset/train.csv')


In [4]:
train

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435
...,...,...,...,...,...,...,...,...,...,...,...
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373


In [5]:
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
#train['dropoff_datetime']=pd.to_datetime(train['dropoff_datetime'],format='%Y-%m-%d %H:%M:%S')

train['pickup_date']= train['pickup_datetime'].dt.date
train['pickup_day']=train['pickup_datetime'].apply(lambda x:x.day)
train['pickup_hour']=train['pickup_datetime'].apply(lambda x:x.hour)
train['pickup_day_of_week']=train['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
# train['dropoff_date']= train['dropoff_datetime'].dt.date
# train['dropoff_day']=train['dropoff_datetime'].apply(lambda x:x.day)
# train['dropoff_hour']=train['dropoff_datetime'].apply(lambda x:x.hour)
# train['dropoff_day_of_week']=train['dropoff_datetime'].apply(lambda x:calendar.day_name[x.weekday()])




In [6]:
def calculateDistance(row):
    R=6373.0 # approximate radius of earth in km
    pickup_lat=radians(row['pickup_latitude'])
    pickup_lon=radians(row['pickup_longitude'])
    dropoff_lat=radians(row['dropoff_latitude'])
    dropoff_lon=radians(row['dropoff_longitude'])
    dlon = dropoff_lon - pickup_lon
    dlat = dropoff_lat - pickup_lat
    a = sin(dlat / 2)**2 + cos(pickup_lat) * cos(dropoff_lat) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [7]:
train['trip_distance']=train.apply(lambda row:calculateDistance(row),axis=1)


In [8]:
#train['trip_duration_in_hour']=train['trip_duration'].apply(lambda x:x/3600)
#test['trip_duration_in_hour']=test['trip_duration'].apply(lambda x:x/3600)

In [9]:
from sklearn.preprocessing import OrdinalEncoder
oc = OrdinalEncoder()
train[['store_and_fwd_flag','pickup_day_of_week']] = oc.fit_transform(train[['store_and_fwd_flag','pickup_day_of_week']])



In [10]:
train = pd.get_dummies(train,columns = ['vendor_id','store_and_fwd_flag','pickup_day_of_week'],prefix_sep = '-',drop_first=True)


In [11]:
train

,id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,pickup_date,pickup_day,pickup_hour,trip_distance,vendor_id-2,store_and_fwd_flag-1.0,pickup_day_of_week-1.0,pickup_day_of_week-2.0,pickup_day_of_week-3.0,pickup_day_of_week-4.0,pickup_day_of_week-5.0,pickup_day_of_week-6.0
0,id2875421,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,455,2016-03-14,14,17,1.498991,1,0,1,0,0,0,0,0
1,id2377394,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,663,2016-06-12,12,0,1.806074,0,0,0,0,1,0,0,0
2,id3858529,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,2124,2016-01-19,19,11,6.387103,1,0,0,0,0,0,1,0
3,id3504673,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,429,2016-04-06,6,19,1.485965,1,0,0,0,0,0,0,1
4,id2181028,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,435,2016-03-26,26,13,1.188962,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458639,id2376096,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,778,2016-04-08,8,13,1.225465,1,0,0,0,0,0,0,0
1458640,id1049543,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,655,2016-01-10,10,7,6.051735,0,0,0,0,1,0,0,0
1458641,id2304944,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,764,2016-04-22,22,6,7.827062,1,0,0,0,0,0,0,0
1458642,id2714485,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,373,2016-01-05,5,15,1.092907,0,0,0,0,0,0,1,0


In [22]:
train_d = train.drop(['pickup_date','pickup_datetime','id','dropoff_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis = 1)



In [13]:
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(train_d.drop('trip_duration',axis =1)))

In [23]:
#train_d = train_d[(z < 2).all(axis=1)]
Q1 = train_d.quantile(0.25)
Q3 = train_d.quantile(0.75)
IQR = Q3 - Q1
train_d = train_d[~((train_d < (Q1 - 1.5 * IQR)) |train_d > (Q3 + 1.5 * IQR)).any(axis=1)]

In [24]:
train_d

,passenger_count,trip_duration,pickup_day,pickup_hour,trip_distance,vendor_id-2,store_and_fwd_flag-1.0,pickup_day_of_week-1.0,pickup_day_of_week-2.0,pickup_day_of_week-3.0,pickup_day_of_week-4.0,pickup_day_of_week-5.0,pickup_day_of_week-6.0
6,4,341,17,22,1.326695,0,0,0,0,0,0,0,0
8,1,255,27,23,1.310765,0,0,0,0,0,0,0,0
12,2,1114,19,9,1.860067,1,0,0,0,0,0,0,0
14,1,1414,27,0,6.384839,1,0,0,0,0,0,0,0
21,1,652,1,12,1.725091,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458622,1,552,19,18,0.729980,1,0,0,0,0,0,0,0
1458629,1,601,24,10,2.560253,1,0,0,0,0,0,0,0
1458636,1,800,12,10,0.930942,1,0,0,0,0,0,0,0
1458639,4,778,8,13,1.225465,1,0,0,0,0,0,0,0


In [26]:
train_d1 = train_d.iloc[:220000]
test_d = train_d.iloc[220000:]
X_train = train_d1.drop(['trip_duration'], axis = 1)
X_test = test_d.drop(['trip_duration'], axis = 1)
y_test = np.log(test_d['trip_duration'].astype('int64'))
y_train =np.log(train_d1['trip_duration'].astype('int64'))

In [27]:
X_test.shape, X_train.shape, y_test.shape, y_train.shape

((2254, 12), (220000, 12), (2254,), (220000,))

In [18]:
# X = train_d.drop(['trip_duration'], axis = 1)
# y = np.log(train_d['trip_duration'].astype('int64'))

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=30)

In [28]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [29]:
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [30]:
subsequences = 2
timesteps = X_train.shape[1]//subsequences
X_train = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))
X_test = X_test.reshape((X_test.shape[0], subsequences, timesteps, 1))
print('Train set shape', X_train.shape)
print('Validation set shape', X_test.shape)

Train set shape (220000, 2, 6, 1)
Validation set shape (2254, 2, 6, 1)


In [31]:
#to stop training after 40 epochs
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=15)
from keras.layers import LSTM

In [ ]:
# model = Sequential()
# model.add(LSTM(12, input_shape=(12, 1)))
# model.add(Dense(231, activation='relu'))
# model.add(Dense(136, activation='relu'))
# model.add(Dense(123, activation='relu'))
# model.add(Dense(216, activation='relu'))
# model.add(Dense(123, activation='relu'))
# model.add(Dense(1))
# # Compile model
# model.compile(loss='mean_squared_error', optimizer='adam')

In [34]:
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=32, kernel_size=1, activation='relu'), input_shape=(None,X_train.shape[2], X_train.shape[3])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(50, activation='relu'))
model_cnn_lstm.add(Dense(36, activation='relu'))
model_cnn_lstm.add(Dense(76, activation='relu'))
model_cnn_lstm.add(Dense(33, activation='relu'))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss='mse', optimizer='adam')

In [35]:
cnn_lstm_history = model_cnn_lstm.fit(X_train, y_train,  validation_data=(X_test, y_test), epochs=10, verbose=1)

Epoch 1/10
6875/6875 [==============================] - 28s 4ms/step - loss: 0.7271 - val_loss: 0.5424
Epoch 2/10
6875/6875 [==============================] - 26s 4ms/step - loss: 0.3803 - val_loss: 0.2620
Epoch 3/10
6875/6875 [==============================] - 26s 4ms/step - loss: 0.3125 - val_loss: 0.2450
Epoch 4/10
6875/6875 [==============================] - 26s 4ms/step - loss: 0.3013 - val_loss: 0.2353
Epoch 5/10
6875/6875 [==============================] - 26s 4ms/step - loss: 0.2949 - val_loss: 0.2359
Epoch 6/10
6875/6875 [==============================] - 26s 4ms/step - loss: 0.2910 - val_loss: 0.2353
Epoch 7/10
6875/6875 [==============================] - 26s 4ms/step - loss: 0.2857 - val_loss: 0.2321
Epoch 8/10
6875/6875 [==============================] - 27s 4ms/step - loss: 0.2829 - val_loss: 0.2287
Epoch 9/10
6875/6875 [==============================] - 27s 4ms/step - loss: 0.2807 - val_loss: 0.2295
Epoch 10/10
6875/6875 [==============================] - 25s 4ms/step - l

In [36]:
# valid_set = (X_test, y_test)
# model.fit(X_train, y_train, batch_size=50, validation_data = valid_set, epochs=10, callbacks=callback, shuffle=True, verbose=1)

In [ ]:

model.save('my_model2.h5') 


In [ ]:
import tensorflow as tf
new_model = tf.keras.models.load_model('my_model2.h5')

In [37]:
prediction = model_cnn_lstm.predict(X_test)


In [38]:
import sklearn.metrics as metrics
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def regression_results(y_true, y_pred):
    
    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    rmse = metrics.mean_squared_error(y_true,y_pred)
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true,y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(rmse),4))
    print('MAPE: ', round(mape,4))
    result = [round(explained_variance,4), round(mean_squared_log_error,4),round(r2,4),round(mean_absolute_error,4),round(mse,4),round(np.sqrt(mse),4)]
    return result

In [39]:
len(prediction),len(y_test)

(2254, 2254)

In [40]:
regression_results(np.exp(y_test),np.exp(prediction).flatten())
#mean_absolute_percentage_error(prediction, y_test)

explained_variance:  0.5352
mean_squared_log_error:  0.222
r2:  0.524
MAE:  273.6801
MSE:  244937.7977
RMSE:  494.9119
MAPE:  38.7343


[0.5352, 0.222, 0.524, 273.6801, 244937.7977, 494.9119]

In [ ]:
result = pd.DataFrame({'y_true':np.exp(y_test),'prediction':np.exp(prediction).flatten()})

In [ ]:
mean_absolute_percentage_error(np.exp(y_test),np.exp(prediction).flatten())

37.448170203469935

In [ ]:
result

,y_true,prediction
310861,417.0,414.137817
922950,22.0,14.014364
833037,434.0,412.354706
456955,1722.0,1703.232666
414925,1750.0,1765.326416
...,...,...
1036921,793.0,765.591187
896940,286.0,269.990479
728018,945.0,936.372498
55969,106.0,92.467026
